# 导入库函数

In [1]:
from __future__ import division
from __future__ import print_function
from gravity_gae.evaluation import compute_scores
from gravity_gae.input_data import load_data
from gravity_gae.model import *
from gravity_gae.optimizer import OptimizerAE, OptimizerVAE
from gravity_gae.preprocessing import *
import numpy as np
import os
import scipy.sparse as sp
import tensorflow as tf
import time

tf.app.flags.DEFINE_string('f', '', 'kernel')#添加的，不报错

# 定义超参数

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.logging.set_verbosity(tf.logging.ERROR)

flags = tf.app.flags
FLAGS = flags.FLAGS

flags.DEFINE_string('dataset', 'Cornell', 'Name of the graph dataset')

# Select machine learning task to perform on graph
flags.DEFINE_string('task', 'task_3', 'Name of the link prediction task')
''' See section 4.1. of paper for details about tasks:

- task_1: General Directed Link Prediction

- task_2: Biased Negative Samples Directed Link Prediction

- task_3: Bidirectionality Prediction
'''

# Model
flags.DEFINE_string('model', 'gravity_gcn_vae', 'Name of the model')
''' Available Models:

- gcn_ae: Graph Autoencoder from Kipf and Welling (2016), with 2-layer
          GCN encoder and inner product decoder

- gcn_vae: Variational Graph Autoencoder from Kipf and Welling (2016), with
           Gaussian priors, 2-layer GCN encoders and inner product decoder

- source_target_gcn_ae: Source-Target Graph Autoencoder, as introduced
                        in section 2.6 of paper, with 2-layer GCN encoder
                        and asymmetric inner product decoder

- source_target_gcn_vae: Source-Target Graph Variational Autoencoder, as
                         introduced in section 2.6, with Gaussian priors,
                         2-layer GCN encoders and asymmetric inner product
 
- gravity_gcn_ae: Gravity-Inspired Graph Autoencoder, as introduced in
                  section 3.3 of paper, with 2-layer GCN encoder and 
                  gravity-inspired asymmetric decoder
 
- gravity_gcn_vae: Gravity-Inspired Graph Variational Autoencoder, as
                   introduced in section 3.4 of paper, with Gaussian 
                   priors, 2-layer GCN encoders and gravity-inspired decoder
'''

# Model parameters
flags.DEFINE_float('dropout', 0., 'Dropout rate (1 - keep probability).')
flags.DEFINE_integer('epochs', 200, 'Number of epochs in training.')
flags.DEFINE_boolean('features', False, 'Include node features or not in GCN')
flags.DEFINE_float('lamb', 1., 'lambda parameter from Gravity AE/VAE models \
                                as introduced in section 3.5 of paper, to \
                                balance mass and proximity terms')
flags.DEFINE_float('learning_rate', 0.1, 'Initial learning rate (with Adam)')
flags.DEFINE_integer('hidden', 32, 'Number of units in GCN hidden layer.')
flags.DEFINE_integer('dimension', 12, 'Dimension of GCN output: \
- equal to embedding dimension for standard AE/VAE and source-target AE/VAE \
- equal to (embedding dimension - 1) for gravity-inspired AE/VAE, as the \
last dimension captures the "mass" parameter tilde{m}')
flags.DEFINE_boolean('normalize', False, 'Whether to normalize embedding \
                                          vectors of gravity models')
flags.DEFINE_float('epsilon', 0.01, 'Add epsilon to distances computations \
                                       in gravity models, for numerical \
                                       stability')
# Experimental setup parameters
flags.DEFINE_integer('nb_run', 10, 'Number of model run + test')
flags.DEFINE_float('prop_val', 5., 'Proportion of edges in validation set \
                                   (for Task 1)')
flags.DEFINE_float('prop_test', 10., 'Proportion of edges in test set \
                                      (for Tasks 1 and 2)')
flags.DEFINE_boolean('validation', False, 'Whether to report validation \
                                           results  at each epoch (for \
                                           Task 1)')
flags.DEFINE_boolean('verbose', True, 'Whether to print comments details.')

In [3]:
# Lists to collect average results
mean_roc = []
mean_ap = []
mean_time = []


# Load graph dataset
if FLAGS.verbose:
    print("Loading data...")
adj_init, features = load_data(FLAGS.dataset)


# The entire training process is repeated FLAGS.nb_run times
for i in range(FLAGS.nb_run):

    # Edge Masking: compute Train/Validation/Test set
    if FLAGS.verbose:
        print("Masking test edges...")

    if FLAGS.task == 'task_1':
        # Edge masking for General Directed Link Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_general_link_prediction(adj_init, FLAGS.prop_test,
                                                FLAGS.prop_val)
    elif FLAGS.task == 'task_2':
        # Edge masking for B.N.S. Directed Link Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_biased_negative_samples(adj_init, FLAGS.prop_test)
    elif FLAGS.task == 'task_3':
        # Edge masking for Bidirectionality Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_bidirectional_link_prediction(adj_init)
    else:
        raise ValueError('Undefined task!')

    # Preprocessing and initialization
    if FLAGS.verbose:
        print("Preprocessing and Initializing...")
    # Compute number of nodes
    num_nodes = adj.shape[0]
    # If features are not used, replace feature matrix by identity matrix
    if not FLAGS.features:
        features = sp.identity(adj.shape[0])
    # Preprocessing on node features
    features = sparse_to_tuple(features)
    num_features = features[2][1]
    features_nonzero = features[1].shape[0]

    # Define placeholders
    placeholders = {
        'features': tf.sparse_placeholder(tf.float32),
        'adj': tf.sparse_placeholder(tf.float32),
        'adj_orig': tf.sparse_placeholder(tf.float32),
        'dropout': tf.placeholder_with_default(0., shape=())
    }

    # Create model
    model = None
    if FLAGS.model == 'gcn_ae':
        # Standard Graph Autoencoder
        model = GCNModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'gcn_vae':
        # Standard Graph Variational Autoencoder
        model = GCNModelVAE(placeholders, num_features, num_nodes,
                            features_nonzero)
    elif FLAGS.model == 'source_target_gcn_ae':
        # Source-Target Graph Autoencoder
        if FLAGS.dimension % 2 != 0:
            raise ValueError('Dimension must be even for Source-Target models')
        model = SourceTargetGCNModelAE(placeholders, num_features,
                                     features_nonzero)
    elif FLAGS.model == 'source_target_gcn_vae':
        # Source-Target Graph Variational Autoencoder
        if FLAGS.dimension % 2 != 0:
            raise ValueError('Dimension must be even for Source-Target models')
        model = SourceTargetGCNModelVAE(placeholders, num_features,
                                      num_nodes, features_nonzero)
    elif FLAGS.model == 'gravity_gcn_ae':
        # Gravity-Inspired Graph Autoencoder
        model = GravityGCNModelAE(placeholders, num_features,
                                  features_nonzero)
    elif FLAGS.model == 'gravity_gcn_vae':
        # Gravity-Inspired Graph Variational Autoencoder
        model = GravityGCNModelVAE(placeholders, num_features, num_nodes,
                                   features_nonzero)
    else:
        raise ValueError('Undefined model!')

    # Optimizer (see tkipf/gae original GAE repository for details)
    pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
    norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0]
                                                - adj.sum()) * 2)
    with tf.name_scope('optimizer'):
        # Optimizer for Non-Variational Autoencoders
        if FLAGS.model in ('gcn_ae', 'source_target_gcn_ae', 'gravity_gcn_ae'):
            opt = OptimizerAE(preds = model.reconstructions,
                              labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                            validate_indices = False), [-1]),
                              pos_weight = pos_weight,
                              norm = norm)
        # Optimizer for Variational Autoencoders
        elif FLAGS.model in ('gcn_vae', 'source_target_gcn_vae', 'gravity_gcn_vae'):
            opt = OptimizerVAE(preds = model.reconstructions,
                               labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                             validate_indices = False), [-1]),
                               model = model,
                               num_nodes = num_nodes,
                               pos_weight = pos_weight,
                               norm = norm)

    # Normalization and preprocessing on adjacency matrix
    adj_norm = preprocess_graph(adj)
    adj_label = sparse_to_tuple(adj + sp.eye(adj.shape[0]))

    # Initialize TF session
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    # Model training
    if FLAGS.verbose:
        print("Training...")
    # Flag to compute total running time
    t_start = time.time()
    for epoch in range(FLAGS.epochs):
        # Flag to compute running time for each epoch
        t = time.time()
        # Construct feed dictionary
        feed_dict = construct_feed_dict(adj_norm, adj_label, features,
                                        placeholders)
        feed_dict.update({placeholders['dropout']: FLAGS.dropout})
        # Weight update
        outs = sess.run([opt.opt_op, opt.cost, opt.accuracy],
                        feed_dict = feed_dict)
        # Compute average loss
        avg_cost = outs[1]
        if FLAGS.verbose:
            # Display epoch information
            print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
                  "time=", "{:.5f}".format(time.time() - t))
            # Validation (implemented for Task 1 only)
            if FLAGS.validation and FLAGS.task == 'task_1':
                feed_dict.update({placeholders['dropout']: 0})
                emb = sess.run(model.z_mean, feed_dict = feed_dict)
                feed_dict.update({placeholders['dropout']: FLAGS.dropout})
                val_roc, val_ap = compute_scores(val_edges, val_edges_false, emb)
                print("val_roc=", "{:.5f}".format(val_roc), "val_ap=", "{:.5f}".format(val_ap))

    # Compute total running time
    mean_time.append(time.time() - t_start)

    # Get embedding from model
    emb = sess.run(model.z_mean, feed_dict = feed_dict)

    # Test model
    if FLAGS.verbose:
        print("Testing model...")
    # Compute ROC and AP scores on test sets
    roc_score, ap_score = compute_scores(test_edges, test_edges_false, emb)
    # Append to list of scores over all runs
    mean_roc.append(roc_score)
    mean_ap.append(ap_score)

# Report final results
print("\nTest results for", FLAGS.model,
      "model on", FLAGS.dataset, "on", FLAGS.task, "\n",
      "___________________________________________________\n")

print("AUC scores\n", mean_roc)
print("Mean AUC score: ", np.mean(mean_roc),
      "\nStd of AUC scores: ", np.std(mean_roc), "\n \n")

print("AP scores \n", mean_ap)
print("Mean AP score: ", np.mean(mean_ap),
      "\nStd of AP scores: ", np.std(mean_ap), "\n \n")

print("Running times\n", mean_time)
print("Mean running time: ", np.mean(mean_time),
      "\nStd of running time: ", np.std(mean_time), "\n \n")

Loading data...
Masking test edges...
Number of bidirectional edges in the graph: 18.0
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.58780 time= 0.10480
Epoch: 0002 train_loss= 1.14401 time= 0.00298
Epoch: 0003 train_loss= 0.83916 time= 0.00501
Epoch: 0004 train_loss= 0.86263 time= 0.00299
Epoch: 0005 train_loss= 0.84047 time= 0.00300
Epoch: 0006 train_loss= 0.69165 time= 0.00301
Epoch: 0007 train_loss= 0.62388 time= 0.00199
Epoch: 0008 train_loss= 0.63602 time= 0.00300
Epoch: 0009 train_loss= 0.64643 time= 0.00200
Epoch: 0010 train_loss= 0.64566 time= 0.00199
Epoch: 0011 train_loss= 0.61636 time= 0.00200
Epoch: 0012 train_loss= 0.58302 time= 0.00200
Epoch: 0013 train_loss= 0.56086 time= 0.00300
Epoch: 0014 train_loss= 0.55374 time= 0.00200
Epoch: 0015 train_loss= 0.55274 time= 0.00200
Epoch: 0016 train_loss= 0.56331 time= 0.00300
Epoch: 0017 train_loss= 0.55035 time= 0.00200
Epoch: 0018 train_loss= 0.53339 time= 0.00200
Epoch: 0019 train_loss= 0.52590 time= 0

Epoch: 0187 train_loss= 0.42119 time= 0.00201
Epoch: 0188 train_loss= 0.43233 time= 0.00200
Epoch: 0189 train_loss= 0.43410 time= 0.00200
Epoch: 0190 train_loss= 0.43059 time= 0.00200
Epoch: 0191 train_loss= 0.43044 time= 0.00200
Epoch: 0192 train_loss= 0.43347 time= 0.00199
Epoch: 0193 train_loss= 0.42357 time= 0.00200
Epoch: 0194 train_loss= 0.43155 time= 0.00200
Epoch: 0195 train_loss= 0.42636 time= 0.00200
Epoch: 0196 train_loss= 0.43199 time= 0.00199
Epoch: 0197 train_loss= 0.42711 time= 0.00200
Epoch: 0198 train_loss= 0.42229 time= 0.00200
Epoch: 0199 train_loss= 0.43145 time= 0.00200
Epoch: 0200 train_loss= 0.43407 time= 0.00199
Testing model...
Masking test edges...
Number of bidirectional edges in the graph: 18.0
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.67665 time= 0.09110
Epoch: 0002 train_loss= 1.30261 time= 0.00296
Epoch: 0003 train_loss= 0.91900 time= 0.00206
Epoch: 0004 train_loss= 0.73864 time= 0.00302
Epoch: 0005 train_loss= 0.83533 time= 

Epoch: 0186 train_loss= 0.42372 time= 0.00200
Epoch: 0187 train_loss= 0.42794 time= 0.00201
Epoch: 0188 train_loss= 0.42354 time= 0.00199
Epoch: 0189 train_loss= 0.42303 time= 0.00200
Epoch: 0190 train_loss= 0.42334 time= 0.00199
Epoch: 0191 train_loss= 0.42112 time= 0.00300
Epoch: 0192 train_loss= 0.42051 time= 0.00200
Epoch: 0193 train_loss= 0.42526 time= 0.00201
Epoch: 0194 train_loss= 0.43185 time= 0.00199
Epoch: 0195 train_loss= 0.42710 time= 0.00301
Epoch: 0196 train_loss= 0.42247 time= 0.00200
Epoch: 0197 train_loss= 0.42499 time= 0.00217
Epoch: 0198 train_loss= 0.41948 time= 0.00283
Epoch: 0199 train_loss= 0.42947 time= 0.00201
Epoch: 0200 train_loss= 0.42476 time= 0.00199
Testing model...
Masking test edges...
Number of bidirectional edges in the graph: 18.0
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.61638 time= 0.11000
Epoch: 0002 train_loss= 1.17874 time= 0.00300
Epoch: 0003 train_loss= 0.81175 time= 0.00198
Epoch: 0004 train_loss= 0.83545 time= 

Epoch: 0165 train_loss= 0.42739 time= 0.00401
Epoch: 0166 train_loss= 0.42781 time= 0.00294
Epoch: 0167 train_loss= 0.43739 time= 0.00201
Epoch: 0168 train_loss= 0.42678 time= 0.00202
Epoch: 0169 train_loss= 0.42542 time= 0.00297
Epoch: 0170 train_loss= 0.42645 time= 0.00302
Epoch: 0171 train_loss= 0.42842 time= 0.00396
Epoch: 0172 train_loss= 0.42043 time= 0.00301
Epoch: 0173 train_loss= 0.43005 time= 0.00197
Epoch: 0174 train_loss= 0.42317 time= 0.00207
Epoch: 0175 train_loss= 0.42851 time= 0.00202
Epoch: 0176 train_loss= 0.42739 time= 0.00300
Epoch: 0177 train_loss= 0.43108 time= 0.00200
Epoch: 0178 train_loss= 0.42145 time= 0.00197
Epoch: 0179 train_loss= 0.42764 time= 0.00197
Epoch: 0180 train_loss= 0.42754 time= 0.00199
Epoch: 0181 train_loss= 0.43307 time= 0.00297
Epoch: 0182 train_loss= 0.43370 time= 0.00419
Epoch: 0183 train_loss= 0.42118 time= 0.00288
Epoch: 0184 train_loss= 0.42608 time= 0.00200
Epoch: 0185 train_loss= 0.42690 time= 0.00301
Epoch: 0186 train_loss= 0.42497 ti

Epoch: 0160 train_loss= 0.43225 time= 0.00195
Epoch: 0161 train_loss= 0.42679 time= 0.00307
Epoch: 0162 train_loss= 0.43169 time= 0.00199
Epoch: 0163 train_loss= 0.43362 time= 0.00294
Epoch: 0164 train_loss= 0.42763 time= 0.00207
Epoch: 0165 train_loss= 0.43010 time= 0.00293
Epoch: 0166 train_loss= 0.43712 time= 0.00201
Epoch: 0167 train_loss= 0.42812 time= 0.00195
Epoch: 0168 train_loss= 0.43014 time= 0.00199
Epoch: 0169 train_loss= 0.42935 time= 0.00295
Epoch: 0170 train_loss= 0.42900 time= 0.00308
Epoch: 0171 train_loss= 0.43318 time= 0.00201
Epoch: 0172 train_loss= 0.43552 time= 0.00199
Epoch: 0173 train_loss= 0.42692 time= 0.00199
Epoch: 0174 train_loss= 0.43084 time= 0.00200
Epoch: 0175 train_loss= 0.42841 time= 0.00206
Epoch: 0176 train_loss= 0.42408 time= 0.00299
Epoch: 0177 train_loss= 0.42522 time= 0.00203
Epoch: 0178 train_loss= 0.42691 time= 0.00206
Epoch: 0179 train_loss= 0.42718 time= 0.00201
Epoch: 0180 train_loss= 0.42621 time= 0.00200
Epoch: 0181 train_loss= 0.42895 ti

...

Epoch: 0145 train_loss= 0.43943 time= 0.00304
Epoch: 0146 train_loss= 0.43302 time= 0.00295
Epoch: 0147 train_loss= 0.43467 time= 0.00296
Epoch: 0148 train_loss= 0.43153 time= 0.00404
Epoch: 0149 train_loss= 0.43737 time= 0.00299
Epoch: 0150 train_loss= 0.43457 time= 0.00212
Epoch: 0151 train_loss= 0.42893 time= 0.00194
Epoch: 0152 train_loss= 0.42671 time= 0.00294
Epoch: 0153 train_loss= 0.43458 time= 0.00304
Epoch: 0154 train_loss= 0.43412 time= 0.00296
Epoch: 0155 train_loss= 0.43495 time= 0.00399
Epoch: 0156 train_loss= 0.43136 time= 0.00301
Epoch: 0157 train_loss= 0.43170 time= 0.00301
Epoch: 0158 train_loss= 0.43522 time= 0.00308
Epoch: 0159 train_loss= 0.43310 time= 0.00287
Epoch: 0160 train_loss= 0.43255 time= 0.00402
Epoch: 0161 train_loss= 0.43480 time= 0.00301
Epoch: 0162 train_loss= 0.43119 time= 0.00294
Epoch: 0163 train_loss= 0.42736 time= 0.00406
Epoch: 0164 train_loss= 0.42974 time= 0.00303
Epoch: 0165 train_loss= 0.43145 time= 0.00290
Epoch: 0166 train_loss= 0.42617 ti

Epoch: 0136 train_loss= 0.43271 time= 0.00301
Epoch: 0137 train_loss= 0.43120 time= 0.00303
Epoch: 0138 train_loss= 0.43703 time= 0.00330
Epoch: 0139 train_loss= 0.44061 time= 0.00264
Epoch: 0140 train_loss= 0.43437 time= 0.00299
Epoch: 0141 train_loss= 0.43296 time= 0.00308
Epoch: 0142 train_loss= 0.43574 time= 0.00196
Epoch: 0143 train_loss= 0.42958 time= 0.00267
Epoch: 0144 train_loss= 0.43344 time= 0.00201
Epoch: 0145 train_loss= 0.42841 time= 0.00189
Epoch: 0146 train_loss= 0.42896 time= 0.00301
Epoch: 0147 train_loss= 0.43623 time= 0.00204
Epoch: 0148 train_loss= 0.43446 time= 0.00300
Epoch: 0149 train_loss= 0.43516 time= 0.00313
Epoch: 0150 train_loss= 0.43538 time= 0.00297
Epoch: 0151 train_loss= 0.43180 time= 0.00299
Epoch: 0152 train_loss= 0.43511 time= 0.00196
Epoch: 0153 train_loss= 0.43767 time= 0.00412
Epoch: 0154 train_loss= 0.43518 time= 0.00293
Epoch: 0155 train_loss= 0.43189 time= 0.00311
Epoch: 0156 train_loss= 0.43458 time= 0.00288
Epoch: 0157 train_loss= 0.43907 ti

Epoch: 0133 train_loss= 0.42968 time= 0.00287
Epoch: 0134 train_loss= 0.43213 time= 0.00306
Epoch: 0135 train_loss= 0.43342 time= 0.00196
Epoch: 0136 train_loss= 0.43757 time= 0.00196
Epoch: 0137 train_loss= 0.44160 time= 0.00308
Epoch: 0138 train_loss= 0.43118 time= 0.00394
Epoch: 0139 train_loss= 0.43072 time= 0.00204
Epoch: 0140 train_loss= 0.43426 time= 0.00300
Epoch: 0141 train_loss= 0.43207 time= 0.00406
Epoch: 0142 train_loss= 0.43060 time= 0.00299
Epoch: 0143 train_loss= 0.43204 time= 0.00301
Epoch: 0144 train_loss= 0.42820 time= 0.00294
Epoch: 0145 train_loss= 0.43118 time= 0.00315
Epoch: 0146 train_loss= 0.42744 time= 0.00484
Epoch: 0147 train_loss= 0.43178 time= 0.00305
Epoch: 0148 train_loss= 0.42735 time= 0.00195
Epoch: 0149 train_loss= 0.43244 time= 0.00203
Epoch: 0150 train_loss= 0.43592 time= 0.00306
Epoch: 0151 train_loss= 0.43280 time= 0.00196
Epoch: 0152 train_loss= 0.42514 time= 0.00303
Epoch: 0153 train_loss= 0.42940 time= 0.00299
Epoch: 0154 train_loss= 0.42745 ti

Epoch: 0127 train_loss= 0.43763 time= 0.00400
Epoch: 0128 train_loss= 0.43623 time= 0.00300
Epoch: 0129 train_loss= 0.43702 time= 0.00200
Epoch: 0130 train_loss= 0.43899 time= 0.00200
Epoch: 0131 train_loss= 0.44956 time= 0.00303
Epoch: 0132 train_loss= 0.44024 time= 0.00303
Epoch: 0133 train_loss= 0.43701 time= 0.00299
Epoch: 0134 train_loss= 0.43582 time= 0.00300
Epoch: 0135 train_loss= 0.43549 time= 0.00195
Epoch: 0136 train_loss= 0.43934 time= 0.00197
Epoch: 0137 train_loss= 0.43808 time= 0.00300
Epoch: 0138 train_loss= 0.43505 time= 0.00306
Epoch: 0139 train_loss= 0.43935 time= 0.00296
Epoch: 0140 train_loss= 0.44173 time= 0.00301
Epoch: 0141 train_loss= 0.43358 time= 0.00400
Epoch: 0142 train_loss= 0.43652 time= 0.00201
Epoch: 0143 train_loss= 0.44230 time= 0.00398
Epoch: 0144 train_loss= 0.43528 time= 0.00304
Epoch: 0145 train_loss= 0.44368 time= 0.00196
Epoch: 0146 train_loss= 0.43814 time= 0.00306
Epoch: 0147 train_loss= 0.43590 time= 0.00194
Epoch: 0148 train_loss= 0.43721 ti

Epoch: 0113 train_loss= 0.44051 time= 0.00401
Epoch: 0114 train_loss= 0.43980 time= 0.00302
Epoch: 0115 train_loss= 0.43729 time= 0.00200
Epoch: 0116 train_loss= 0.43536 time= 0.00304
Epoch: 0117 train_loss= 0.43466 time= 0.00303
Epoch: 0118 train_loss= 0.42944 time= 0.00301
Epoch: 0119 train_loss= 0.42954 time= 0.00193
Epoch: 0120 train_loss= 0.43255 time= 0.00302
Epoch: 0121 train_loss= 0.43991 time= 0.00306
Epoch: 0122 train_loss= 0.43388 time= 0.00296
Epoch: 0123 train_loss= 0.43407 time= 0.00196
Epoch: 0124 train_loss= 0.43687 time= 0.00175
Epoch: 0125 train_loss= 0.43131 time= 0.00196
Epoch: 0126 train_loss= 0.43164 time= 0.00200
Epoch: 0127 train_loss= 0.43589 time= 0.00207
Epoch: 0128 train_loss= 0.43347 time= 0.00295
Epoch: 0129 train_loss= 0.42735 time= 0.00302
Epoch: 0130 train_loss= 0.43252 time= 0.00302
Epoch: 0131 train_loss= 0.43744 time= 0.00304
Epoch: 0132 train_loss= 0.44496 time= 0.00189
Epoch: 0133 train_loss= 0.43430 time= 0.00201
Epoch: 0134 train_loss= 0.43674 ti